### Step 1: Data Preprocessing
1. **Load the Data:**
   - Load the news data and the three CSV files into your preferred data analysis tool (e.g., Python using pandas).

2. **Clean the Data:**
   - Handle missing values, remove duplicates, and correct any inconsistencies.
   - For the news data, focus on the `content` column for text analysis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline
import seaborn as sns

import itertools
import re
import string
import pickle
import os

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.model_selection import KFold

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.corpus import stopwords, wordnet
from wordcloud import WordCloud
from copy import deepcopy

from IPython.display import (
    Markdown as md,
    Latex,
    HTML,
)

from tqdm.auto import tqdm
import json

# set plot style
sns.set()

from google.colab import drive

drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
player_detail = pd.read_csv('/content/drive/MyDrive/nlp data/Copy of player_detail.csv')
transfer_history = pd.read_csv('/content/drive/MyDrive/nlp data/Copy of tm_player_transfer_history.csv')
player_market_value_development = pd.read_csv('/content/drive/MyDrive/nlp data/Copy of tm_player_market_value_development.csv')

miao_feed_all_docs = pd.read_csv('/content/drive/MyDrive/nlp data/Data/miao_feed_all_docs.csv')
news_feed_all_docs = pd.read_csv('/content/drive/MyDrive/nlp data/Data/news_feed_all_docs.csv')

<ipython-input-2-4ffd8b7aca70>:5: DtypeWarning: Columns (8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  miao_feed_all_docs = pd.read_csv('/content/drive/MyDrive/nlp data/Data/miao_feed_all_docs.csv')


In [ ]:
miao_feed_all_docs.head(2)

,url,title,author,content,date,title_image,spider_type,summary,mini_summary_cn,keywords,summary_cn,title_cn,mini_summary,file_name,chunks,translations,video_generated
0,https://theathletic.com/4427670/2023/04/19/pio...,Stefano Pioli praises Rafael Leao following AC...,['Omar Garrick'],"<html><body><div><div class=""sc-294a6039-6 fbD...",2023-04-19,https://cdn.theathletic.com/cdn-cgi/image/widt...,theathletic,AC Milan head coach Stefano Pioli says Rafael ...,AC米兰主教练斯特凡诺·皮奥利表示拉斐尔·雷奥正在成为一名“冠军”，在这名边锋的帮助下，米兰...,"['napoli', 'ac milan', 'inter milan', 'olivier...",AC米兰主教练斯特凡诺·皮奥利表示拉斐尔·雷奥正在成为一名“冠军”，在这名边锋的帮助下，米兰...,史蒂芬诺·皮奥利(Stefano Pioli) 对拉斐尔·莱昂(Rafael Leao) 的...,AC Milan head coach Stefano Pioli says Rafael ...,NaN,NaN,NaN,NaN
1,https://www.telegraph.co.uk/football/2023/04/1...,Todd Boehly and Co have nowhere left to go aft...,['Sam Wallace'],"<html><body><div><div class=""component article...",2023-04-19,https://www.telegraph.co.uk/content/dam/footba...,telegraph,[image]Behdad Eghbali and Todd Boehly - Todd B...,托德·博厄利(Todd Boehly)和贝赫达德·艾格拜利(Behdad Eghbali)等...,"['carlo ancelotti', 'kai havertz', 'conor gall...",图片描述：Behdad Eghbali 和 Todd Boehly - Todd Boehl...,托德·博勒(Todd Boehly)等人在切尔西(Chelsea)再次失败后无路可走,[image]Behdad Eghbali and Todd Boehly - Todd B...,NaN,NaN,NaN,NaN


In [ ]:
miao_feed_all_docs.shape

(77847, 17)

In [ ]:
miao_feed_all_docs.isna().sum()

url                    0
title                  0
author                 0
content                0
date                  91
title_image         2506
spider_type            0
summary             5382
mini_summary_cn    63063
keywords            5382
summary_cn         63062
title_cn            5409
mini_summary       12392
file_name           9092
chunks             13654
translations       13654
video_generated    71852
dtype: int64

In [ ]:
# Drop multiple columns
miao_feed_all_docs_dropped = miao_feed_all_docs.drop(columns=[
    'mini_summary_cn', 'summary_cn', 'title_cn',
    'file_name', 'chunks', 'translations', 'video_generated', 'mini_summary', 'title_image', 'summary'])

In [ ]:
miao_feed_all_docs_dropped.isna().sum()

url               0
title             0
author            0
content           0
date             91
spider_type       0
keywords       5382
dtype: int64

In [ ]:
news_feed_all_docs.head(2)

,categories,url,title,content,author,date,spider_type,file_name,video_generated
0,['Manchester City'],https://www.mirror.co.uk/sport/football/news/p...,10 most expensive Premier League keepers with ...,"<div id=""readability-page-1"" class=""page""><div...",['Matthew Cooper'],2023-09-25 14:31:18+00:00,newsnow,2023-09-25-ff8bd382227d41a2daf30561fc952e31,NaN
1,['Manchester United'],https://metro.co.uk/2023/09/25/gran-74-left-sh...,Gran in crash with Marcus Rashford had no idea...,"<div id=""readability-page-1"" class=""page""><div...",['Liam Coleman'],2023-09-25 15:58:13+00:00,newsnow,2023-09-25-8983ec6c0529664e948ee63d3df06887,NaN


In [ ]:
# Drop multiple columns
news_feed_all_docs_dropped = news_feed_all_docs.drop(
    columns=['video_generated', 'file_name'])

In [ ]:
news_feed_all_docs_dropped.isna().sum()

categories      0
url             0
title          23
content         0
author          0
date            0
spider_type     0
dtype: int64

In [ ]:
def clean_content(text):
    # Remove HTML tags
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)

    # Remove "[image]" from the text
    text = re.sub(r'\[image\]', '', text)

    # Remove "(Photo: ...)" where "photo:" is case insensitive
    text = re.sub(r'\(photo:\s*[^)]*\)', '', text, flags=re.IGNORECASE)

    # Remove URLs
    url_pattern = r'https?://\S+|www\.\S+'
    text = re.sub(url_pattern, '', text)

    # Remove excessive values in between text (adjust as per your specific case)
    text = re.sub(r'\b(?:excessive_value1|excessive_value2)\b', '', text, flags=re.IGNORECASE)

    clean_text = re.sub(r'\s+', ' ', text).strip()

    return clean_text

In [ ]:
miao_feed_all_docs_dropped['content'] = miao_feed_all_docs_dropped['content'].apply(clean_content)
news_feed_all_docs_dropped['content'] = news_feed_all_docs_dropped['content'].apply(clean_content)

In [ ]:
miao_feed_all_docs_dropped.head(2)

,url,title,author,content,date,spider_type,keywords
0,https://theathletic.com/4427670/2023/04/19/pio...,Stefano Pioli praises Rafael Leao following AC...,['Omar Garrick'],AC Milan head coach Stefano Pioli says Rafael ...,2023-04-19,theathletic,"['napoli', 'ac milan', 'inter milan', 'olivier..."
1,https://www.telegraph.co.uk/football/2023/04/1...,Todd Boehly and Co have nowhere left to go aft...,['Sam Wallace'],Behdad Eghbali and Todd Boehly - Todd Boehly a...,2023-04-19,telegraph,"['carlo ancelotti', 'kai havertz', 'conor gall..."


In [ ]:
miao_feed_all_docs_dropped.to_csv('/content/drive/MyDrive/nlp data/Data/miao_feed_all_docs_dropped.csv', index=False)
news_feed_all_docs_dropped.to_csv('/content/drive/MyDrive/nlp data/Data/news_feed_all_docs_dropped.csv', index=False)